CSCE 4143: Data Mining

Group 7 Final Project

UFC Stuff

https://www.kaggle.com/datasets/akshaysinghim/ufc-fight-data-1993-to-2023

Levi Crider, Spencer Smith, Caleb Holmes, and Ethan Weems

## Installing Dependencies

In [207]:
%pip install numpy
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [208]:
import pandas as pd
# Load data
df = pd.read_csv("data/ufc_data.csv")

In [209]:
df = df.rename(columns={'R_fighter': 'RED'})
df = df.rename(columns={'B_fighter': 'BLUE'})
df = df.rename(columns={'R_SIG_STR_pct': 'R_SIG_PCT'})
df = df.rename(columns={'B_SIG_STR_pct': 'B_SIG_PCT'})
df = df.rename(columns={'R_TD_pct': 'R_TD_PCT'})
df = df.rename(columns={'B_TD_pct': 'B_TD_PCT'})
df = df.rename(columns={'win_by': 'WIN_TYPE'})
df = df.rename(columns={'last_round': 'LAST_RD'})
df = df.rename(columns={'last_round_time': 'LAST_RD_TIME'})
df = df.rename(columns={'Format': 'NUM_RD'})
df = df.rename(columns={'Referee': 'REF'})
df = df.rename(columns={'date': 'DATE'})
df = df.rename(columns={'location': 'LOC'})
df = df.rename(columns={'Fight_type': 'TYPE'})
df = df.rename(columns={'Winner': 'WINNER'})


# Cleaning Data

In [210]:
# Dropping all rows with missing values
df.replace("---", pd.NaT, inplace=True)
df.dropna()



# Dropping all rows before may of 2001
df['DATE'] = pd.to_datetime(df['DATE'], format="%B %d, %Y").dt.strftime("%Y-%m-%d")
limit_date = '2001-04-01'
df = df[(df['DATE'] > limit_date)]



In [211]:
# Remove percentage signs from columns
df = df.replace('%', '', regex=True)

In [212]:
# Combine last round and last round time into one column called total_time

# Convert LAST_RD to time elapsed in rounds before final round
df['LAST_RD'] = (df['LAST_RD'].astype(int) * 5) - 5
df['LAST_RD'] = pd.to_timedelta(df['LAST_RD'], unit='m')

# Convert LAST_RD_TIME to format acceptable by timedelta
df['LAST_RD_TIME'] = '00:' + df['LAST_RD_TIME'].astype(str)
df['LAST_RD_TIME'] = pd.to_timedelta(df['LAST_RD_TIME'])

# Add times
df['TOTAL_TIME'] = df['LAST_RD'] + df['LAST_RD_TIME']

# Reformat
df['TOTAL_TIME'] = df['TOTAL_TIME'].apply(lambda x: '{:02}:{:02}'.format(x.components.minutes, x.components.seconds))

# Drop old columns
df = df.drop(columns=['LAST_RD', 'LAST_RD_TIME'])
df

,RED,BLUE,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_PCT,B_SIG_PCT,R_TOTAL_STR.,B_TOTAL_STR.,...,R_GROUND,B_GROUND,WIN_TYPE,NUM_RD,REF,DATE,LOC,TYPE,WINNER,TOTAL_TIME
0,Aljamain Sterling,Sean O'Malley,0,1,17 of 35,25 of 35,48,71,24 of 46,26 of 36,...,0 of 0,14 of 18,KO/TKO,5 Rnd (5-5-5-5-5),Marc Goddard,2023-08-19,"Boston, Massachusetts, USA",UFC Bantamweight Title Bout,Sean O'Malley,05:51
1,Chris Weidman,Brad Tavares,0,0,37 of 92,70 of 115,40,60,39 of 94,71 of 116,...,0 of 0,0 of 0,Decision - Unanimous,3 Rnd (5-5-5),Keith Peterson,2023-08-19,"Boston, Massachusetts, USA",Middleweight Bout,Brad Tavares,15:00
2,Neil Magny,Ian Garry,0,0,27 of 66,91 of 150,40,60,51 of 93,111 of 171,...,0 of 0,3 of 5,Decision - Unanimous,3 Rnd (5-5-5),Keith Peterson,2023-08-19,"Boston, Massachusetts, USA",Welterweight Bout,Ian Garry,15:00
3,Marlon Vera,Pedro Munhoz,0,0,141 of 251,113 of 246,56,45,141 of 251,113 of 246,...,0 of 1,0 of 0,Decision - Unanimous,3 Rnd (5-5-5),Herb Dean,2023-08-19,"Boston, Massachusetts, USA",Bantamweight Bout,Marlon Vera,15:00
4,Zhang Weili,Amanda Lemos,1,0,163 of 217,24 of 63,75,38,296 of 358,29 of 68,...,96 of 122,0 of 0,Decision - Unanimous,5 Rnd (5-5-5-5-5),Kevin MacDonald,2023-08-19,"Boston, Massachusetts, USA",UFC Women's Strawweight Title Bout,Zhang Weili,25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7013,Tony DeSouza,Steve Berger,0,0,28 of 48,2 of 18,58,11,51 of 71,108 of 129,...,25 of 39,0 of 0,Decision - Unanimous,3 Rnd (5-5-5),Mario Yamasaki,2001-05-04,"Atlantic City, New Jersey, USA",Welterweight Bout,Tony DeSouza,15:00
7014,BJ Penn,Joey Gilbert,0,0,22 of 32,3 of 7,68,42,76 of 86,11 of 15,...,16 of 21,0 of 0,KO/TKO,3 Rnd (5-5-5),John McCarthy,2001-05-04,"Atlantic City, New Jersey, USA",Lightweight Bout,BJ Penn,04:57
7015,Shonie Carter,Matt Serra,2,0,18 of 33,10 of 25,54,40,49 of 65,27 of 44,...,2 of 11,2 of 2,KO/TKO,3 Rnd (5-5-5),Mario Yamasaki,2001-05-04,"Atlantic City, New Jersey, USA",Welterweight Bout,Shonie Carter,14:51
7016,Semmy Schilt,Pete Williams,2,0,15 of 26,6 of 9,57,66,22 of 34,14 of 18,...,2 of 2,5 of 6,KO/TKO,3 Rnd (5-5-5),John McCarthy,2001-05-04,"Atlantic City, New Jersey, USA",Heavyweight Bout,Semmy Schilt,06:28


# Decision Tree Classifier